In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importing the dataset
dataset = pd.read_csv('train_features.csv',encoding = "ISO-8859-1",nrows=50000)
X = dataset.iloc[:, [5,10,11,12,13,14,15,16,17,20,21,23,24,25,26]].values
y = dataset.iloc[:, 31].values